In [5]:
import numpy as np
import librosa
from scipy.io import wavfile
from glob import glob
import re
import warnings
from numpy.random import random, uniform, randint, choice

In [4]:
class DataGenerator(object):
    def __init__(self, input_dir):

        self.debug = False
        self._input_files = None
        self._cached_waves = {}

        # msg normalization params
        # inspired by https://github.com/fchollet/keras/blob/master/keras/preprocessing/image.py#L529
        self.samplewise_norm = True
        # following two needs to be computed in many msgs
        self.msg_mean = None
        self.msg_std = None

        self.input_dir = input_dir
        self.sample_rate = 16000
        self.duration = 1.  # sec
        self.n_fft = 512
        self.n_mels = 64
        self.msg_w = 256

        # valid labels
        self.valid_labels = set(LABELS)

        self.silence_files = glob(self.input_dir + '/_background_noise_/*.wav')

        # random mixing settings

        self.mix_with = {}

        self.mix_with[self.input_dir +
                      '/_background_noise_/doing_the_dishes.wav'] = {
                          'volume': [0.05, 0.75],
                          'probability': 0.2
                      }

        self.mix_with[self.input_dir +
                      '/_background_noise_/exercise_bike.wav'] = {
                          'volume': [0.05, 0.75],
                          'probability': 0.2
                      }

        self.mix_with[self.input_dir +
                      '/_background_noise_/white_noise.wav'] = {
                          'volume': [0.001, 0.06],
                          'probability': 0.2
                      }

        self.mix_with[self.input_dir +
                      '/_background_noise_/dude_miaowing.wav'] = {
                          'volume': [0.05, 0.75],
                          'probability': 0.2
                      }

        self.mix_with[self.input_dir +
                      '/_background_noise_/pink_noise.wav'] = {
                          'volume': [0.001, 0.06],
                          'probability': 0.2
                      }

        self.mix_with[self.input_dir +
                      '/_background_noise_/running_tap.wav'] = {
                          'volume': [0.25, 0.75],
                          'probability': 0.2
                      }

        # random transforms

        self.transforms = {
            'pitch': {
                'probability': 0.5,
                'range': [-10., 10.]
            },
            'speed': {
                'probability': 0.33,
                'range': [0.5, 2.]
            },
            'volume': {
                'probability': 0.25,
                'range': [0.5, 1.25]
            }
        }

        # silence-specific transforms
        self.silence_transforms = {
            'volume': {
                'probability': 1.,
                'range': [0.01, 1.]
            }
        }

    # compute mel-scaled spectrogram
    def msg(self, wave):

        hop_length = int(1 + self.duration * self.sample_rate //
                         (self.msg_w - 1))
        desired_wave_len = int(hop_length * (self.msg_w - 1))

        # pad wave if neccessary to get the desired msg width
        if desired_wave_len > len(wave):
            wave = np.pad(
                wave, (0, desired_wave_len - len(wave)),
                'constant',
                constant_values=(0.))

        # trim wave if it's too long
        elif len(wave) > desired_wave_len:
            wave = wave[:desired_wave_len]

        msg = librosa.feature.melspectrogram(
            y=wave,
            sr=self.sample_rate,
            hop_length=hop_length,
            n_fft=self.n_fft,
            n_mels=self.n_mels)
        msg = librosa.logamplitude(msg**2, ref_power=1.)
        assert msg.shape[1] == self.msg_w

        return msg.astype(np.float32)

    @property
    def input_files(self):
        if self._input_files is None:

            def _get_label(path):
                m = re.findall('audio/([^/]+)/', path)
                if not m: return None
                if m[0] in self.valid_labels: return m[0]
                else: return 'unknown'

            ff = glob(self.input_dir + '/**/*.wav', recursive=True)
            ff = filter(lambda x: '_background_noise_/' not in x, ff)

            self._input_files = {}

            for f in ff:
                label = _get_label(f)
                if label not in self._input_files:
                    self._input_files[label] = [f]
                else:
                    self._input_files[label].append(f)

        return self._input_files

    # read file and extract random segment
    def _load_random_segment(self, file):

        if file not in self._cached_waves:
            # supress warnings
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                sr, wave = wavfile.read(file)
                wave = wave.astype(np.float32)
            self._cached_waves[file] = (sr, wave)

        sr, wave = self._cached_waves[file]
        assert sr == self.sample_rate
        l = int(sr * self.duration)

        start = randint(0, len(wave) - l)
        wave = wave[start:start + l]

        return wave

    # gen random audio sample for 'silence' label
    def generate_silence_audio(self):
        silence_file = choice(self.silence_files)
        if self.debug: print('silence file', silence_file)
        wave = self._load_random_segment(silence_file)
        wave = self.apply_transforms(wave, silence_transforms=True)
        return wave

    # gen random audio for non-silence labels
    def generate_audio(self):
        # pick label
        label = choice(list(self.valid_labels))

        if 'silence' == label:
            return self.generate_silence_audio(), label
        else:
            # pick wav file
            file = choice(self.input_files[label])

            if self.debug: print('file', file)

            sr, wave = wavfile.read(file)
            wave = wave.astype(np.float32)
            assert sr == self.sample_rate

            # pad too short ones/trim too long

            desired_len = int(self.duration * self.sample_rate)
            len_diff = len(wave) - desired_len

            if len_diff < 0:  # too short
                wave = np.pad(wave, (-len_diff + 1) // 2,
                              'median')[:desired_len]
            elif len_diff > 0:  # too long
                wave = wave[len_diff // 2:desired_len + len_diff // 2]

            assert len(wave) == desired_len

            # transforms
            wave = self.apply_transforms(wave)\

            return wave, label

    # apply randon transforms
    def apply_transforms(self, wave, silence_transforms=False):

        if silence_transforms:
            # silence/volume
            t = self.silence_transforms['volume']
            if random() < t['probability']:
                factor = uniform(*t['range'])
                wave = self.transform_volume(wave, factor)

        # mix
        for file, options in self.mix_with.items():
            if random() < options['probability']:
                wave2 = self._load_random_segment(file)
                volume2 = uniform(*options['volume'])

                if self.debug:
                    print('mixing with %s at %.2f vol' % (file, volume2))

                wave = self.mix(wave, 1., wave2, volume2)

        # pitch
        t = self.transforms['pitch']
        if random() < t['probability']:
            wave = self.transform_pitch(wave, uniform(*t['range']))

        # speed
        t = self.transforms['speed']
        if random() < t['probability']:
            wave = self.transform_speed(wave, uniform(*t['range']))

        return wave

    def transform_pitch(self, wave, factor):
        if self.debug: print('transforming pitch', factor)
        return librosa.effects.pitch_shift(wave, self.sample_rate, factor)

    def transform_volume(self, wave, factor):
        if self.debug: print('transforming volume', factor)
        return np.multiply(wave, factor)

    def transform_speed(self, wave, factor):
        if self.debug: print('transforming speed', factor)

        orig_len = len(wave)
        wave = librosa.effects.time_stretch(wave, factor)
        len_diff = len(wave) - orig_len

        # pad/trim from the center
        if len_diff < 0:
            wave = np.pad(wave, (-len_diff + 1) // 2, 'median')[:orig_len]
        elif len_diff > 0:
            wave = wave[len_diff // 2:orig_len + len_diff // 2]

        assert len(wave) == orig_len
        return wave

    def mix(self, wave1, volume1, wave2, volume2):
        if volume1 + volume2 > 0:
            volume1, volume2 = \
                volume1 / (volume1 + volume2), \
                volume2 / (volume1 + volume2)
            return wave1 * volume1 + wave2 * volume2

    def normalize_msg(self, msg):

        if self.msg_mean is not None:
            msg -= self.msg_mean

        if self.msg_std is not None:
            msg /= self.msg_std + 1e-7

        if self.samplewise_norm:
            msg -= np.mean(msg)
            msg /= np.std(msg) + 1e-7

        return msg

    def compute_msg_norm_params(self, n_steps=100):
        msgs = np.zeros((n_steps, self.n_mels, self.msg_w), dtype=np.float32)

        for i in range(n_steps):
            msgs[i] = self.msg(self.generate_audio()[0])

        self.msg_mean = np.mean(msgs)
        self.msg_std = np.std(msgs)